In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['USER_AGENT'] = 'myagent'
os.environ['COHERE_API_KEY'] = os.getenv("COHERE_API_KEY")

In [5]:
from tqdm.auto import tqdm

In [2]:
# dependencies
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import ChatCohere
from langchain.chains import ConversationalRetrievalChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\souga\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [6]:
# Store splits
vectorstore = FAISS.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings(model_name = "shibing624/text2vec-base-multilingual"))
print("[INFO] Finished building KB.", end="\n")

[INFO] Finished building KB.


In [7]:
# LLM
llm = ChatCohere()

## LCEL (Langchain Expression Language)

In [27]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [28]:
condense_question_system_template = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

In [31]:
condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [32]:
history_aware_retriever = create_history_aware_retriever(
    llm, vectorstore.as_retriever(), condense_question_prompt
)

In [33]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [34]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [35]:
qa_chain = create_stuff_documents_chain(llm, qa_prompt)

In [36]:
convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [37]:
lcel_results = convo_qa_chain.invoke(
    {
        "input": "What are autonomous agents?",
        "chat_history": [],
    }
)